In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk.data

nltk.download() 
import logging
from gensim.models import word2vec


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
my_fpath_courses = "../Data/main_coursera.csv"

course_data = pd.read_csv(my_fpath_courses)

course_data.columns

In [ ]:
def clean(course_desc, remove_stopwords=True):
    # 1. Remove HTML
    course_desc = BeautifulSoup(course_desc).get_text()
    #  
    # 2. Remove non-letters
    course_desc = re.sub("[^a-zA-Z]"," ", course_desc)
    #
    # 3. Convert words to lower case and split them
    words = course_desc.lower().split()
    #
    # 4. Optionally remove stop words (True by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)


In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a course descriptions into parsed sentences
def course_desc_to_sentences( course_desc, tokenizer, remove_stopwords=True ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(course_desc.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call clean to get a list of words
            sentences.append( clean( raw_sentence,remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [ ]:
sentences = []  # Initialize an empty list of sentences
print("Parsing sentences from training set")
for course_desc in course_data['Course Description'].values:
    sentences += course_desc_to_sentences(str(course_desc), tokenizer)

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers,size=num_features, min_count = min_word_count, window = context, sample = downsampling)

model.init_sims(replace=True)
model_name = "300features_40minwords_10context_courses"
model.save(model_name)

In [ ]:

sim = model.n_similarity(['data','science'],['data'])

print(sim)